In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import seaborn as sns
import datetime as dt

from ipywidgets import interact, interactive, fixed
import ipywidgets as widgets
from IPython.display import display

from sklearn import linear_model
import warnings
warnings.filterwarnings('ignore')
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.metrics import accuracy_score as acs

__Importing Data__

In [ ]:
#Cases APIs
case_time_series=pd.read_csv("https://api.covid19india.org/csv/latest/case_time_series.csv")
case_time_series["Date_YMD"]=pd.to_datetime(case_time_series.Date_YMD,format="%Y-%m-%d")
case_time_series.index=case_time_series['Date_YMD']
#display(case_time_series)
#display(case_time_series.describe())
#display(case_time_series.shape)

#Testing APIs
tested_numbers_icmr_data=pd.read_csv("https://api.covid19india.org/csv/latest/tested_numbers_icmr_data.csv")
#display(tested_numbers_icmr_data.tail())
#tested_numbers_icmr_data.info()

#Create new dataframe for testing data
testingAndVaccinating=pd.DataFrame()
testingAndVaccinating["Date"]=tested_numbers_icmr_data["Tested As Of"]
testingAndVaccinating["Daily Tested"]=tested_numbers_icmr_data["Daily RTPCR Samples Collected_ICMR Application"]
testingAndVaccinating["Total Tested"]=tested_numbers_icmr_data["Total Samples Tested"]
testingAndVaccinating["Total Vaccinations"]=tested_numbers_icmr_data["Total Individuals Vaccinated"]
testingAndVaccinating["Date"]=pd.to_datetime(testingAndVaccinating.Date,infer_datetime_format=True)
testingAndVaccinating.index=testingAndVaccinating["Date"]
testingAndVaccinating.drop(["Date"],axis=1,inplace=True)
testingAndVaccinating.fillna(0,axis=1,inplace=True)
#display(testingAndVaccinating)

# __India Daily Analysis__

In [ ]:
#Cases
TC=case_time_series["Total Confirmed"]
TR=case_time_series["Total Recovered"]
TD=case_time_series["Total Deceased"]
X=case_time_series["Daily Confirmed"]
Y=case_time_series["Daily Recovered"]
Z=case_time_series["Daily Deceased"]
D=case_time_series["Date_YMD"]
DF=case_time_series["Date"]
TOTC=case_time_series["Total Confirmed"]
TOTR=case_time_series["Total Recovered"]
TOTD=case_time_series["Total Deceased"]

#Testing and jabs
T=testingAndVaccinating["Daily Tested"]
TV=testingAndVaccinating["Total Vaccinations"]
Tdate=testingAndVaccinating.index
TOTT=testingAndVaccinating["Total Tested"]

DAC=(X-Y-Z)  #Daily Active cases
TAC=(TC-TR-TD) #Total Active cases
AR=(TC-TR-TD)/TC*100  #Active cases ratio
dr=(TD/TC*100) #Death Rate
rr=(TR/TC*100) #Recovery Rate
gr = (X.pct_change(periods=7)*100)/7 #Growth Rate

#Daily change in stats
delC=X.diff()
delR=Y.diff()
delD=Z.diff()
delA=DAC.diff()
delT=T.diff()

#Moving Averages
case_time_series['CMA'] = X.rolling(window=7).mean()
case_time_series['RMA'] = Y.rolling(window=7).mean()
case_time_series['DMA'] = Z.rolling(window=7).mean()
case_time_series['AMA'] = DAC.rolling(window=7).mean()
testingAndVaccinating["TMA"] = T.rolling(window=7).mean()
CMA=case_time_series["CMA"]
RMA=case_time_series["RMA"]
AMA=case_time_series["AMA"]
DMA=case_time_series["DMA"]
TMA=testingAndVaccinating["TMA"]

Atoday=X[len(X)-1]-Y[len(Y)-1]-Z[len(Z)-1]
TOTA=TOTC-TOTR-TOTD

# Visualization of data

In [ ]:
#Confirmed Cases
confirmed = go.Figure(data=[go.Bar(x=D,y=X,    #Trace 1
                                   name="Daily Cases",
                                   marker = dict(color = 'rgba(255, 0, 0,1)'),
                                   text="Peak: {}, Average: {}".format(X.max(),round(X.mean()),2)),
                            go.Scatter(x=D, y=CMA,
                                   mode='lines',
                                   name="7-Day moving average",
                                   line_color="red")]
                     )
confirmed.add_annotation(text="{}:<br><b>{}</b>  ∆= {} <br>Total: {}"
                         .format(DF[len(DF)-1],X[len(X)-1],delC[len(delC)-1],TOTC[len(TOTC)-1]),
                         bgcolor='rgba(255, 0, 0,0.2)',
                         font=dict(size=16,family="orbitron"),
                         borderwidth=2,
                         bordercolor="red",
                         x=0, 
                         xanchor="left",
                         xref="paper",
                         y=1,
                         yanchor="top",
                         yref="paper",
                         showarrow=False,
                         align="left" 
                  )
confirmed.update_layout(
    margin=dict(l=20,r=20,b=50,t=70),
    title="Confirmed cases",
    xaxis_title="Date",
    yaxis_title="Cases",
    legend_title="India: ",
    paper_bgcolor='rgba(255, 0, 0,0.3)',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([dict(count=3,step="month",stepmode="backward"),
                          dict(count=6,step="month",stepmode="backward"),
                          dict(count=1,step="year",stepmode="backward")])
        ),
        rangeslider=dict(visible=True),
    )
)

#Recovered Cases
recovered = go.Figure(data=[go.Bar(x=D,y=Y,    #Trace 1
                                   name="Daily recoveries",
                                   marker = dict(color = 'rgba(0, 255, 0,1)'),
                                   text="Peak: {}, Average: {}".format(Y.max(),round(Y.mean()),2)),
                            go.Scatter(x=D, y=RMA,
                                   mode='lines',
                                   name="7-Day moving average",
                                   line_color="green")]
                     )

recovered.add_annotation(text="{}:<br><b>{}</b>  ∆= {} <br>Total: {}"
                         .format(DF[len(DF)-1],Y[len(Y)-1],delR[len(delR)-1],TOTR[len(TOTR)-1]),
                         bgcolor='rgba(0, 255, 0,0.2)',
                         font=dict(size=16,family="orbitron"),
                         borderwidth=2,
                         bordercolor="green",
                         x=0, 
                         xanchor="left",
                         xref="paper",
                         y=1,
                         yanchor="top",
                         yref="paper",
                         showarrow=False,
                         align="left"
                        )
recovered.update_layout(
    margin=dict(l=20,r=20,b=50,t=70),
    title="Recoveries",
    xaxis_title="Date",
    yaxis_title="Recoveries",
    legend_title="India: ",
    paper_bgcolor='rgba(0, 255, 0,0.3)',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([dict(count=3,step="month",stepmode="backward"),
                          dict(count=6,step="month",stepmode="backward"),
                          dict(count=1,step="year",stepmode="backward")])
        ),
        rangeslider=dict(visible=True),
    )
)

#Active Cases
active = go.Figure(data=[go.Bar(x=D,y=DAC,    #Trace 1
                                   name="Daily active cases",
                                   marker = dict(color = 'rgba(0, 0, 255,1)'),
                                   text="Peak: {}, Average: {}".format(DAC.max(),round(DAC.mean()),2)),
                            go.Scatter(x=D, y=AMA,
                                   mode='lines',
                                   name="7-Day moving average",
                                   line_color="blue")]
                     )
active.add_annotation(text="{}:<br><b>{}</b>  ∆= {} <br>Total: {}"
                      .format(DF[len(DF)-1],Atoday,delA[len(delA)-1],TOTA[len(TOTA)-1]),
                         bgcolor='rgba(0, 0, 255,0.2)',
                         font=dict(size=16,family="orbitron"),
                         borderwidth=2,
                         bordercolor="blue",
                         x=0, 
                         xanchor="left",
                         xref="paper",
                         y=1,
                         yanchor="top",
                         yref="paper",
                         showarrow=False,
                         align="left"
                        )
active.update_layout(
    margin=dict(l=20,r=20,b=50,t=70),
    title="Active Cases",
    xaxis_title="Date",
    yaxis_title="Cases",
    legend_title="India: ",
    paper_bgcolor='rgba(0, 0, 255,0.3)',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([dict(count=3,step="month",stepmode="backward"),
                          dict(count=6,step="month",stepmode="backward"),
                          dict(count=1,step="year",stepmode="backward")])
        ),
        rangeslider=dict(visible=True),
    )
)

#Deaths 
deaths = go.Figure(data=[go.Bar(x=D,y=Z,    #Trace 1
                                   name="Daily deaths",
                                   marker = dict(color = 'rgba(0, 0, 0,1)'),
                                   text="Peak: {}, Average: {}".format(Z.max(),round(Z.mean()),2)),
                            go.Scatter(x=D, y=DMA,
                                   mode='lines',
                                   name="7-Day moving average",
                                   line_color="black")]
                     )
deaths.add_annotation(text="{}:<br><b>{}</b>  ∆= {}".format(DF[len(DF)-1],Z[len(Z)-1],delD[len(delD)-1]),
                         bgcolor='rgba(0, 0, 0,0.2)',
                         font=dict(size=16,family="orbitron"),
                         borderwidth=2,
                         bordercolor="black",
                         x=0, 
                         xanchor="left",
                         xref="paper",
                         y=1,
                         yanchor="top",
                         yref="paper",
                         showarrow=False,
                         align="left"
                        )
deaths.update_layout(
    margin=dict(l=20,r=20,b=50,t=70),
    title="Deceased",
    xaxis_title="Date",
    yaxis_title="Deaths",
    legend_title="India: ",
    paper_bgcolor='rgba(0, 0, 0,0.3)',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([dict(count=3,step="month",stepmode="backward"),
                          dict(count=6,step="month",stepmode="backward"),
                          dict(count=1,step="year",stepmode="backward")])
        ),
        rangeslider=dict(visible=True),
    )
)
#Rates
rates = go.Figure(make_subplots(rows=3, cols=2,
                      specs=[[{}, {}],[{"colspan": 2}, None],[{"colspan":2},None]],
                      subplot_titles=("Active Rate", "Recovery Rate","Death Rate","Growth Rate"))
                 )

rates.add_trace(
    go.Scatter(x=D,y=AR,
               mode="lines",
               name="Active rate",
               marker = dict(color = 'rgba(0, 0, 255,1)'),
               text="Maximum: {}%, Average: {}%".format(round(AR.max(),2),round(AR.mean()),2)),
    row=1,col=1
)
rates.add_trace(
    go.Scatter(x=D,y=rr,
               mode="lines",
               name="Recovery rate",
               marker = dict(color = 'rgba(0, 255, 0,1)'),
               text="Maximum: {}%, Average: {}%".format(round(rr.max(),2),round(rr.mean()),2)),
    row=1,col=2
)
rates.add_trace(
    go.Scatter(x=D,y=dr,
               mode="lines",
               name="Death rate",
               marker = dict(color = 'rgba(0, 0, 0,1)'),
               text="Maximum: {}%, Average: {}%".format(round(dr.max(),2),round(dr.mean()),2)),
    row=2,col=1
)
rates.add_trace(
    go.Scatter(x=D[70:],y=gr[70:],
               mode="lines",
               name="Cases growth rate",
               marker = dict(color = 'rgba(255, 0, 0,1)'),
               text="Maximum: {}%, Average: {}%".format(round(gr[70:].max(),2),round(gr[70:].mean()),2)),
    row=3,col=1
)
rates.update_xaxes(title_text="Date", row=1, col=1)
rates.update_xaxes(title_text="Date", row=1, col=2)
rates.update_yaxes(title_text="Rate", row=1, col=1)
rates.update_yaxes(title_text="Rate", row=1, col=2)
rates.update_xaxes(title_text="Date", row=2, col=1)
rates.update_yaxes(title_text="Rate", row=2, col=1)
rates.update_xaxes(title_text="Date", row=3, col=1)
rates.update_yaxes(title_text="Rate", row=3, col=1)

rates.update_layout(
    title_text="Statistical Rates",
    legend_title="India: ",
    paper_bgcolor='rgba(0, 100, 100,0.5)',
    margin=dict(l=20,r=20,b=50,t=70),
    height=600
)

#Testing
testing = go.Figure(data=[go.Bar(x=Tdate,y=T,    #Trace 1
                                   name="Daily Testing",
                                   marker = dict(color = 'rgba(128, 0, 128,1)'),
                                   text="Peak: {}, Average: {}".format(T.max(),round(T.mean()),2)),
                            go.Scatter(x=testingAndVaccinating.index, y=TMA,
                                   mode='lines',
                                   name="7-Day moving average",
                                   line_color="purple")]
                     )
testing.add_annotation(text="{}:<br><b>{}</b>  ∆= {}".format(Tdate[len(Tdate)-1],T[len(T)-1],delT[len(delT)-1]),
                         bgcolor='rgba(128, 0, 128,0.2)',
                         font=dict(size=16,family="orbitron"),
                         borderwidth=2,
                         bordercolor="purple",
                         x=0, 
                         xanchor="left",
                         xref="paper",
                         y=1,
                         yanchor="top",
                         yref="paper",
                         showarrow=False,
                         align="left" 
                  )
testing.update_layout(
    margin=dict(l=20,r=20,b=50,t=70),
    title="Testing Data",
    xaxis_title="Date",
    yaxis_title="Tests",
    legend_title="India: ",
    paper_bgcolor='rgba(128, 0, 128,0.3)',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([dict(count=3,step="month",stepmode="backward"),
                          dict(count=6,step="month",stepmode="backward"),
                          dict(count=1,step="year",stepmode="backward")])
        ),
        rangeslider=dict(visible=True),
    )
)

confirmed.show()
recovered.show()
active.show()
deaths.show()
testing.show()
rates.show()

# LSTM model to predict curve

In [ ]:
case_time_series1=case_time_series.sort_index(ascending=True,axis=0)
new_dataset=pd.DataFrame(index=range(0,len(case_time_series)),columns=['Date_YMD','Daily Confirmed'])

for i in range(0,len(case_time_series1)):
    new_dataset["Date_YMD"][i]=case_time_series1['Date_YMD'][i]
    new_dataset["Daily Confirmed"][i]=case_time_series1["Daily Confirmed"][i]
    
#Normalize the Dataset
scaler=MinMaxScaler(feature_range=(0,1))
new_dataset.index=new_dataset.Date_YMD
new_dataset.drop("Date_YMD",axis=1,inplace=True)
final_dataset=new_dataset.values

train_data=final_dataset[0:(len(final_dataset)-70),:]
valid_data=final_dataset[(len(final_dataset)-70):,:]
scaled_data=scaler.fit_transform(final_dataset)

x_train_data,y_train_data=[],[]

for i in range(70,len(train_data)):
    x_train_data.append(scaled_data[i-70:i,0])
    y_train_data.append(scaled_data[i,0])
    
x_train_data,y_train_data=np.array(x_train_data),np.array(y_train_data)
x_train_data=np.reshape(x_train_data,(x_train_data.shape[0],x_train_data.shape[1],1))

#Build and train the LSTM model
lstm_model=Sequential()
lstm_model.add(LSTM(units=70,return_sequences=True,input_shape=(x_train_data.shape[1],1)))
lstm_model.add(LSTM(units=70))
lstm_model.add(Dense(1))

inputs_data=new_dataset[len(new_dataset)-len(valid_data)-70:].values
inputs_data=inputs_data.reshape(-1,1)
inputs_data=scaler.transform(inputs_data)

lstm_model.compile(loss='mean_squared_error',optimizer='adam')
lstm_model.fit(x_train_data,y_train_data,epochs=1,batch_size=1,verbose=2)

In [ ]:
#Testing data for the model
X_test=[]
for i in range(70,inputs_data.shape[0]):
    X_test.append(inputs_data[i-70:i,0])
X_test=np.array(X_test)

X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
predicted_daily_confirmed=lstm_model.predict(X_test)
predicted_daily_confirmed=scaler.inverse_transform(predicted_daily_confirmed)

train_data=new_dataset[:(len(final_dataset)-70)]
valid_data=new_dataset[(len(final_dataset)-70):]
valid_data['Predictions']=predicted_daily_confirmed

#Confirmed Cases
plt.figure(figsize=(16,8))
plt.plot(train_data["Daily Confirmed"])
plt.plot(valid_data['Daily Confirmed'],label = "Real Value")
plt.plot(valid_data["Predictions"],label = "Prediction")

plt.legend()
plt.grid()
plt.show()

# **__State Wise analysis__**

In [ ]:
#Cases
state_wise_daily = pd.read_csv("https://api.covid19india.org/csv/latest/state_wise_daily.csv")
state_wise_daily["Date_YMD"]=pd.to_datetime(state_wise_daily.Date_YMD,format="%Y-%m-%d")
state_wise=pd.read_csv("https://api.covid19india.org/csv/latest/state_wise.csv")
#display(state_wise)
#display(state_wise.info())
#display(state_wise_daily)
#display(state_wise_daily.info())

#Testing and jabs
statewise_tested_numbers_data=pd.read_csv("https://api.covid19india.org/csv/latest/statewise_tested_numbers_data.csv")
statewise_tested_numbers_data["Updated On"]=pd.to_datetime(statewise_tested_numbers_data["Updated On"],
                                                           infer_datetime_format=True)
statewise_tested_numbers_data["State"]=statewise_tested_numbers_data["State"].replace(
    ["Maharashtra","Kerala","Karnataka","Andhra Pradesh","Tamil Nadu","Delhi","Uttar Pradesh","West Bengal","Odisha",
     "Rajasthan","Chhattisgarh","Telangana","Haryana","Gujarat","Bihar","Madhya Pradesh","Assam","Punjab","Jammu and Kashmir",
     "Jharkhand","Uttarakhand","Himachal Pradesh","Goa","Puducherry","Tripura","Manipur","Chandigarh",'Arunachal Pradesh',
     'Meghalaya',"Nagaland","Ladakh","Sikkim","Andaman and Nicobar Islands","Mizoram","Dadra and Nagar Haveli and Daman and Diu",
     "Lakshadweep"],
    ["MH","KL","KA","AP","TN","DL","UP","WB","OR","RJ","CT","TG","HR","GJ","BR","MP","AS","PB","JK","JH","UT","HP","GA","PY",
     "TR","MN","CH","AR","ML","NL","LA","SK","AN","MZ","DN","LD"])
statewise_tested_numbers_data.index=statewise_tested_numbers_data['Updated On']
#display(statewise_tested_numbers_data)state_wise_daily = pd.read_csv("https://api.covid19india.org/csv/latest/state_wise_daily.csv")
state_wise_daily["Date_YMD"]=pd.to_datetime(state_wise_daily.Date_YMD,format="%Y-%m-%d")
state_wise=pd.read_csv("https://api.covid19india.org/csv/latest/state_wise.csv")
#display(state_wise.head())
#display(state_wise.info())
#display(state_wise_daily)
#display(state_wise_daily.info())

In [ ]:
def f(x):
    display(x)
    return x

C = state_wise_daily.columns[3:]
#print("Select a state to view statistics.")
P = interactive(f, x=widgets.Dropdown(options=C,value='MH',description='State Code:',disabled=False))
print("Select a state to view statistics.")
display(P)

In [ ]:
State = P.result
X=state_wise_daily[state_wise_daily.Status=="Confirmed"][State]
Y=state_wise_daily[state_wise_daily.Status=="Recovered"][State]
Z=state_wise_daily[state_wise_daily.Status=="Deceased"][State]
AC=X.values-Y.values-Z.values
Active=pd.DataFrame(AC,columns=["Active"])
DF=state_wise_daily[state_wise_daily.Status=="Confirmed"]["Date"]
D=state_wise_daily[state_wise_daily.Status=="Confirmed"]["Date_YMD"]
TOTC=state_wise[state_wise["State_code"]==State]["Confirmed"]
TOTR=state_wise[state_wise["State_code"]==State]["Recovered"]
TOTD=state_wise[state_wise["State_code"]==State]["Deaths"]
TOTA=TOTC-TOTR-TOTD

#Testing
Date=statewise_tested_numbers_data[statewise_tested_numbers_data.State==State]["Updated On"]
tested=statewise_tested_numbers_data[statewise_tested_numbers_data.State==State]["Total Tested"].diff()
TOTT=statewise_tested_numbers_data[statewise_tested_numbers_data.State==State]["Total Tested"]

#gr = (X.pct_change(periods=7)*100)/7
#pr=(X.values)/(tested.values)*100

#Latest values
Ctoday=X[3*(len(X))-3]
Rtoday=Y[3*(len(Y))-2]
Dtoday=Z[3*(len(Z))-1]
Ttoday=tested[len(tested)-2]
Atoday=Ctoday-Rtoday-Dtoday

#Daily change in stats
delC=X.diff()
delR=Y.diff()
delD=Z.diff()
delA=Active["Active"].diff()
delT=tested.diff()

#Moving Averages
state_wise_daily['CMA'] = X.rolling(window=7).mean()
state_wise_daily['RMA'] = Y.rolling(window=7).mean()
state_wise_daily['DMA'] = Z.rolling(window=7).mean()
Active['AMA'] = Active["Active"].rolling(window=7).mean()
statewise_tested_numbers_data["TMA"]=tested.rolling(window=7).mean()
CMA=state_wise_daily[state_wise_daily.Status=="Confirmed"]["CMA"]
RMA=state_wise_daily[state_wise_daily.Status=="Recovered"]["RMA"]
DMA=state_wise_daily[state_wise_daily.Status=="Deceased"]["DMA"]
AMA=Active["AMA"]
TMA=statewise_tested_numbers_data[statewise_tested_numbers_data.State==State]["TMA"]

In [ ]:
#Confirmed Cases
STconfirmed = go.Figure(data=[go.Bar(x=D,y=X,    #Trace 1
                                   name="Daily Cases",
                                   marker = dict(color = 'rgba(255, 0, 0,0.7)'),
                                   text="Peak: {}, Average: {}".format(X.max(),round(X.mean()),2)),
                            go.Scatter(x=D, y=CMA,
                                   mode='lines',
                                   name="7-Day moving average",
                                   line_color="red")]
                     )
STconfirmed.add_annotation(text="{}:<br><b>{}</b>  ∆= {} <br>Total: {}"
                           .format(DF[3*(len(DF))-3],Ctoday,delC[3*(len(delC))-3],TOTC.values),
                           bgcolor='rgba(255, 0, 0,0.2)',
                           font=dict(size=16,family="orbitron"),
                           borderwidth=2,
                           bordercolor="red",
                           x=0, 
                           xanchor="left",
                           xref="paper",
                           y=1,
                           yanchor="top",
                           yref="paper",
                           showarrow=False,
                           align="left"
                  )
STconfirmed.update_layout(
    margin=dict(l=20,r=20,b=50,t=70),
    title="State confirmed cases",
    xaxis_title="Date",
    yaxis_title="Cases",
    legend_title="{}: ".format(State),
    paper_bgcolor='rgba(255, 0, 0,0.3)',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([dict(count=3,step="month",stepmode="backward"),
                          dict(count=6,step="month",stepmode="backward"),
                          dict(count=1,step="year",stepmode="backward")])
        ),
        rangeslider=dict(visible=True),
    )
)

#Recovered Cases
STrecovered = go.Figure(data=[go.Bar(x=D,y=Y,    #Trace 1
                                   name="Daily recoveries",
                                   marker = dict(color = 'rgba(0, 255, 0,0.7)'),
                                   text="Peak: {}, Average: {}".format(Y.max(),round(Y.mean()),2)),
                            go.Scatter(x=D, y=RMA,
                                   mode='lines',
                                   name="7-Day moving average",
                                   line_color="green")]
                     )
STrecovered.add_annotation(text="{}:<br><b>{}</b>  ∆= {} <br>Total: {}"
                           .format(DF[3*(len(DF))-3],Rtoday,delR[3*(len(delR))-2],TOTR.values),
                           bgcolor='rgba(0, 255, 0,0.2)',
                           font=dict(size=16,family="orbitron"),
                           borderwidth=2,
                           bordercolor="green",
                           x=0, 
                           xanchor="left",
                           xref="paper",
                           y=1,
                           yanchor="top",
                           yref="paper",
                           showarrow=False,
                           align="left"
                  )
STrecovered.update_layout(
    margin=dict(l=20,r=20,b=50,t=70),
    title="State recoveries",
    xaxis_title="Date",
    yaxis_title="Recoveries",
    legend_title="{}: ".format(State),
    paper_bgcolor='rgba(0, 255, 0,0.3)',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([dict(count=3,step="month",stepmode="backward"),
                          dict(count=6,step="month",stepmode="backward"),
                          dict(count=1,step="year",stepmode="backward")])
        ),
        rangeslider=dict(visible=True),
    )
)

#Active Cases
STactive = go.Figure(data=[go.Bar(x=D,y=Active["Active"],    #Trace 1
                                   name="Daily active",
                                   marker = dict(color = 'rgba(0, 0, 255,0.7)'),
                                   text="Peak: {}, Average: {}".format(Active["Active"].max(),round(Active["Active"].mean()),2)),
                            go.Scatter(x=D, y=AMA,
                                   mode='lines',
                                   name="7-Day moving average",
                                   line_color="blue")
                          ]
                     )
STactive.add_annotation(text="{}:<br><b>{}</b>  ∆= {} <br>Total: {}"
                        .format(DF[3*(len(DF))-3],Atoday,delA[len(delA)-1],TOTA.values),
                           bgcolor='rgba(0, 0, 255,0.2)',
                           font=dict(size=16,family="orbitron"),
                           borderwidth=2,
                           bordercolor="blue",
                           x=0, 
                           xanchor="left",
                           xref="paper",
                           y=1,
                           yanchor="top",
                           yref="paper",
                           showarrow=False,
                           align="left"
                  )
STactive.update_layout(
    margin=dict(l=20,r=20,b=50,t=70),
    title="State active",
    xaxis_title="Date",
    yaxis_title="Active",
    legend_title="{}: ".format(State),
    paper_bgcolor='rgba(0, 0, 255,0.3)',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([dict(count=3,step="month",stepmode="backward"),
                          dict(count=6,step="month",stepmode="backward"),
                          dict(count=1,step="year",stepmode="backward")])
        ),
        rangeslider=dict(visible=True),
    )
)

#Deaths 
STdeaths = go.Figure(data=[go.Bar(x=D,y=Z,    #Trace 1
                                   name="Daily deaths",
                                   marker = dict(color = 'rgba(0, 0, 0,0.7)'),
                                   text="Peak: {}, Average: {}".format(Z.max(),round(Z.mean()),2)),
                            go.Scatter(x=D, y=DMA,
                                   mode='lines',
                                   name="7-Day moving average",
                                   line_color="black")]
                     )
STdeaths.add_annotation(text="{}:<br><b>{}</b>  ∆= {} <br>Total: {}"
                        .format(DF[3*(len(DF))-3],Dtoday,delD[3*(len(delD))-1],TOTD.values),
                           bgcolor='rgba(0, 0, 0,0.2)',
                           font=dict(size=16,family="orbitron"),
                           borderwidth=2,
                           bordercolor="black",
                           x=0, 
                           xanchor="left",
                           xref="paper",
                           y=1,
                           yanchor="top",
                           yref="paper",
                           showarrow=False,
                           align="left"
                  )
STdeaths.update_layout(
    margin=dict(l=20,r=20,b=50,t=70),
    title="State deceased",
    xaxis_title="Date",
    yaxis_title="Deaths",
    legend_title="{}: ".format(State),
    paper_bgcolor='rgba(0, 0, 0,0.3)',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([dict(count=3,step="month",stepmode="backward"),
                          dict(count=6,step="month",stepmode="backward"),
                          dict(count=1,step="year",stepmode="backward")])
        ),
        rangeslider=dict(visible=True),
    )
)

#Testing
STTesting=go.Figure(data=[go.Bar(x=Date,y=tested,    #Trace 1
                                   name="Daily Tests",
                                   marker = dict(color = 'rgba(128, 0, 128,0.7)'),
                                   text="Peak: {}, Average: {}".format(tested.max(),round(tested.mean()),2)),
                            go.Scatter(x=Date, y=TMA,
                                   mode='lines',
                                   name="7-Day moving average",
                                   line_color="purple")]
                     )
STTesting.add_annotation(text="{}:<br><b>{}</b>  ∆= {} <br>Total: {}"
                         .format(Date[len(Date)-2],Ttoday,delT[len(delT)-2],TOTT[len(TOTT)-2]),
                           bgcolor='rgba(128, 0, 128,0.2)',
                           font=dict(size=16,family="orbitron"),
                           borderwidth=2,
                           bordercolor="purple",
                           x=0, 
                           xanchor="left",
                           xref="paper",
                           y=1,
                           yanchor="top",
                           yref="paper",
                           showarrow=False,
                           align="left"
                  )
STTesting.update_layout(
    margin=dict(l=20,r=20,b=50,t=70),
    title="State testing",
    xaxis_title="Date",
    yaxis_title="Tests",
    legend_title="{}: ".format(State),
    paper_bgcolor='rgba(128, 0, 128,0.3)',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([dict(count=3,step="month",stepmode="backward"),
                          dict(count=6,step="month",stepmode="backward"),
                          dict(count=1,step="year",stepmode="backward")])
        ),
        rangeslider=dict(visible=True),
    )
)

STconfirmed.show()
STrecovered.show()
STactive.show()
STdeaths.show()
STTesting.show()

__Active Rate in states__

In [ ]:
SWC=state_wise[state_wise.State!="Total"]["Confirmed"]
SWR=state_wise[state_wise.State!="Total"]["Recovered"]
SWD=state_wise[state_wise.State!="Total"]["Deaths"]
T=state_wise[state_wise.State=="Total"]
SC=state_wise[state_wise.State!="Total"]["State_code"]
ar=((SWC-SWR-SWD)/SWC*100)

#Display
PR = go.Figure(data=[go.Bar(x=SC,y=SWC,    #Trace 1
                                   name="Confirmed",
                            text=SWC,
                                   marker = dict(color = 'rgba(255, 0, 0,1)'),
                           ),
                     go.Bar(x=SC,y=SWR,name="Recovered",
                            text=SWR,
                            marker=dict(color = 'rgba(0, 255, 0,1)')),
                     go.Bar(x=SC,y=SWD,name="Deaths",
                            text=SWD,
                            marker=dict(color = 'rgba(0, 0, 0,1)')),
                    ]
                    
              )
PR.update_traces(texttemplate='%{text:.2s}', textposition='outside')
PR.update_layout(
    margin=dict(l=20,r=20,b=50,t=70),
    title="State-wise",
    xaxis_title="State",
    yaxis_title="Confirmed//Recovered//Deaths",
    legend_title="Plots: ",
    paper_bgcolor='#9ef7ad',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([dict(count=3,step="month",stepmode="backward"),
                          dict(count=6,step="month",stepmode="backward"),
                          dict(count=1,step="year",stepmode="backward")])
        ),
        rangeslider=dict(visible=True),
    )
)
AR = go.Figure(data=[go.Bar(x=SC,y=ar,    #Trace 1
                                   name="Active Rate",
                            text=ar,
                                   marker = dict(color = '#9ef7ad'),
                           )]    
              )
AR.update_traces(texttemplate='%{text:.2s}%', textposition='outside')
AR.update_layout(
    margin=dict(l=20,r=20,b=50,t=70),
    title="State-wise",
    xaxis_title="State",
    yaxis_title="Active Rate",
    legend_title="Plots: ",
    paper_bgcolor='rgba(0, 0, 255,0.3)',
    xaxis=dict(
        rangeselector=dict(
            buttons=list([dict(count=3,step="month",stepmode="backward"),
                          dict(count=6,step="month",stepmode="backward"),
                          dict(count=1,step="year",stepmode="backward")])
        ),
        rangeslider=dict(visible=True),
    )
)
PR.show()
AR.show()